In [62]:
import hashlib
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sn
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [63]:
hosts=["147.32.84.165","147.32.84.191","147.32.84.192","147.32.84.193","147.32.84.204","147.32.84.205",\
       "147.32.84.206","147.32.84.207","147.32.84.208","147.32.84.209"]
infectedHost = hosts[2]

In [72]:
CTU_NETFLOW_FILEPATH ="capture20110818.pcap.netflow.labeled"

COLUMNS = ["Date flow start","Durat","Prot","Src IP Addr:Port","Dst IP Addr:Port","Flags","Tos","Packets Bytes","Flows","Label","Labels"]#,"Src Port","Dst Port"]

In [69]:
def streamFunction(amount_of_packets,CTU_NETFLOW_FILEPATH):
    packets =[]
    with open(CTU_NETFLOW_FILEPATH,"r") as netflow:
            #File is an Argus Unidirectional file created with ra -r so not rly csv. 
            #Let's parse by hand.
            for i,packet  in enumerate(netflow):
                #skip header
                if i == 0:
                    continue
                packet = packet.rstrip('\n')
                split_packet = packet.split("\t")
                #print(len(split_packet))
                if len(split_packet) == 1 :
                    split_packet = packet.split(" ")
                    split_packet[0]=split_packet[0]+" "+split_packet[1]
                    del split_packet[1]
                    
                if "->" in split_packet:
                    split_packet.remove("->")

                if "" in split_packet:
                    split_packet.remove("")

                
                try:
                ## Take all the ports from the ips
                    if ":" in split_packet[3]:
                        split_packet[3]= split_packet[3].split(":")[0]
                    if ":" in split_packet[4]:
                        split_packet[4]= split_packet[4].split(":")[0]
                except Exception as inst:
                    print(i)
                    print(split_packet)
                
                split_packet = [e for e in split_packet if e]
                 ## select packets where one of the hosts(determined previously') is interacting

                if split_packet[3] in hosts: #== infectedHost:
                    packets.append(split_packet)
                elif split_packet[4] in hosts : # == infectedHost:
                    packets.append(split_packet)
                
               # if i == amount_of_packets:
                #    break
    return packets

In [70]:
amount_of_packets= 19000
dfStream = streamFunction(amount_of_packets,CTU_NETFLOW_FILEPATH)

In [73]:
netflowDF=pd.DataFrame(dfStream,columns=COLUMNS)

In [74]:
netflowDF

,Date flow start,Durat,Prot,Src IP Addr:Port,Dst IP Addr:Port,Flags,Tos,Packets Bytes,Flows,Label,Labels
0,2011-08-18 10:20:40.906,0.000,TCP,58.188.108.203,147.32.84.208,S_,0,1,66,1,Botnet
1,2011-08-18 10:37:58.448,0.000,UDP,147.32.84.165,147.32.80.9,INT,0,1,64,1,Botnet
2,2011-08-18 10:37:58.448,0.000,UDP,147.32.80.9,147.32.84.165,INT,0,1,139,1,Botnet
3,2011-08-18 10:38:00.695,0.000,UDP,147.32.84.165,147.32.80.9,INT,0,1,87,1,Botnet
4,2011-08-18 10:38:00.705,0.000,UDP,147.32.80.9,147.32.84.165,INT,0,1,503,1,Botnet
5,2011-08-18 10:38:00.706,0.043,TCP,147.32.84.165,74.125.232.206,SRPA_,0,4,629,1,Botnet
6,2011-08-18 10:38:00.714,0.032,TCP,74.125.232.206,147.32.84.165,SPA_,0,3,253,1,Botnet
7,2011-08-18 10:38:03.255,0.000,UDP,147.32.84.165,147.32.80.9,INT,0,1,76,1,Botnet
8,2011-08-18 10:38:03.266,0.000,UDP,147.32.80.9,147.32.84.165,INT,0,1,403,1,Botnet
9,2011-08-18 10:38:03.267,2.990,UDP,147.32.84.165,65.55.56.40,INT,0,2,180,1,Botnet


# PACKET LEVEL

# HOST LEVEL

In [44]:
features

,Unnamed: 0,Durat,ProtIndex,Flows,Packets Bytes,FlagIndex,Label,DuratDisc,FlowsDisc,PacketsDisc
1,1,0.0,0,139,1,0,0,0,0,0


In [14]:
# PACKET 
# amount of packets transferred by each IP address
# average amount of bytes transfered

TP, FP, FN, TN = 0, 0, 0, 0
X_train,X_test,y_train,y_test = train_test_split(features,target,test_size=0.2,random_state=43)



,Date flow start,Durat,Prot,Src IP Addr:Port,Dst IP Addr:Port,Flags,Tos,Packets Bytes,Flows,Label,Labels,Src Port,Dst Port
0,2011-08-18 10:55:03.872,0.000,UDP,147.32.84.192,147.32.80.9,INT,0,1,64,1,Botnet,1025,53
1,2011-08-18 10:55:03.872,0.000,UDP,147.32.80.9,147.32.84.192,INT,0,1,139,1,Botnet,53,1025
2,2011-08-18 10:55:06.144,0.000,UDP,147.32.84.192,147.32.80.9,INT,0,1,87,1,Botnet,1025,53
3,2011-08-18 10:55:06.154,0.000,UDP,147.32.80.9,147.32.84.192,INT,0,1,503,1,Botnet,53,1025
4,2011-08-18 10:55:06.155,0.153,TCP,147.32.84.192,74.125.232.206,SRPA_,0,4,629,1,Botnet,1027,80
5,2011-08-18 10:55:06.163,0.027,TCP,74.125.232.206,147.32.84.192,SPA_,0,3,253,1,Botnet,80,1027
6,2011-08-18 10:55:08.897,0.000,UDP,147.32.84.192,147.32.80.9,INT,0,1,76,1,Botnet,1025,53
7,2011-08-18 10:55:08.898,0.000,UDP,147.32.80.9,147.32.84.192,INT,0,1,403,1,Botnet,53,1025
8,2011-08-18 10:55:08.901,3.003,UDP,147.32.84.192,65.55.56.40,INT,0,2,180,1,Botnet,123,123
9,2011-08-18 11:05:56.383,0.000,UDP,147.32.84.192,147.32.80.9,INT,0,1,76,1,Botnet,1025,53


In [ ]:
# Host 


In [ ]:
# Machine learning algorith
#SPLIT DATA